# HackerEarth Machine Learning Challenge
## Predict the employee burn out rate

**ABOUT CHALLENGE** <br>
> *To win in the marketplace, you must first win in the workplace.*
~ Doug Conant

> And rightly so! Happy and healthy employees are indisputably more productive at work, and in turn, help the business flourish profoundly.

> However, since working from home has become the new strange normal, over 69% of employees have been showing burnout symptoms globally (source: Monster poll). This rate of burnout is indeed alarming and the likes of Google and Cisco (even HackerEarth!) are taking steps that ensure wellness and reduce burnout among employees.

> On the occasion of World Mental Health Day this October 10th, your organization has stepped in to help its employees plan their weekly calendars appropriately so as to prevent burning out. Your task, as a Machine Learning engineer, is to build a Machine Learning model that predicts burnout rate based on numerous factors such as WFH setup, resources, mental fatigue score, and the like.


***
**Participant:** [Mert Gül](https://www.linkedin.com/in/gulmert89/)<br>
Hackathon Link: [HackerEarth](https://www.hackerearth.com/challenges/competitive/hackerearth-machine-learning-challenge-predict-burnout-rate/)<br>
More information on the dataset: [Kaggle](https://www.kaggle.com/blurredmachine/are-your-employees-burning-out)
***

# 1\. Introduction

Let's go over the steps quickly and get the party started:<br>
* Import the essential modules and the dataset (duh!).
* Check out the NaN and unique values and also the value types.
* Perform one-hot encoding if necessary to make things easier for the models.
  * ...because there are a couple of binary features as well as some discrete & continuous ones.
* Modify the features to get some useful information out of them.
  * Feature engineering, e.g how about we inspect WFH option by gender?
* Fill in the blanks (NaNs) with proper values.
  * Though we might just get rid of them as well.
* Create a baseline model to see where we are.
* Try different models (with different features) and tune their hyperparameters
* Dance with the best model till sunrise.

Dataset:
* **Employee ID**: The unique ID allocated for each employee (example: fffe390032003000)
* **Date of Joining**: The date-time when the employee has joined the organization (example: 2008-12-30)
* **Gender**: The gender of the employee (Male/Female)
* **Company Type**: The type of company where the employee is working (Service/Product)
* **WFH Setup Available**: Is the work from home facility available for the employee (Yes/No)
* **Designation**: The designation of the employee of work in the organization.
    * In the range of [0.0, 5.0] bigger is higher designation.
* **Resource Allocation**: The amount of resource allocated to the employee to work, ie. number of working hours.
    * In the range of [1.0, 10.0] (higher means more resource)
* **Mental Fatigue Score**: The level of fatigue mentally the employee is facing.
    * In the range of [0.0, 10.0] where 0.0 means no fatigue and 10.0 means completely fatigue.
* **Burn Rate**: The value we need to predict for each employee telling the rate of Bur out while working.
    * In the range of [0.0, 1.0] where the higher the value is more is the burn out.

## 1.1 Imports & Dataset

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv("../input/are-your-employees-burning-out/train.csv")
test = pd.read_csv("../input/are-your-employees-burning-out/test.csv")

***
# 2\. Preprocessing

## 2.1 Examine The Dataset

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
# number of unique values per variable
train.nunique()

In [ ]:
# number of empty values
train.isnull().sum()

In [ ]:
# number of empty values (test)
test.isnull().sum()

## 2.2 Processing The Variables

In [ ]:
# cosmetic configurations
sns.set_style(style="whitegrid")

title_font = {"family":"sans-serif", 
              "weight":"bold", 
              "color":"darkgreen", 
              "size":16}

axis_font = {"family":"serif", 
             "weight":"normal", 
             "color":"darkgreen", 
             "size":14}

### 2.2.1 One-Hot Encoding

It would be useful to convert these features,
* Gender
* Company Type
* WFH Setup Available<br>

to binary values by **one-hot encoding**.

#### 2.2.1.1 Train Set

In [ ]:
is_male = pd.get_dummies(train.Gender, drop_first=True)
is_service = pd.get_dummies(train["Company Type"], drop_first=True)
wfh_available = pd.get_dummies(train["WFH Setup Available"], drop_first=True)

In [ ]:
for loc, column in enumerate(["is_male", "is_service", "wfh_available"], start=2):
    train.insert(loc=loc, column=column, value=eval(column))

train.drop(columns=["Gender", "Company Type", "WFH Setup Available"], axis=1, inplace=True)

In [ ]:
train.head()

#### 2.2.1.2 Test Set

In [ ]:
is_male = pd.get_dummies(test.Gender, drop_first=True)
is_service = pd.get_dummies(test["Company Type"], drop_first=True)
wfh_available = pd.get_dummies(test["WFH Setup Available"], drop_first=True)

for loc, column in enumerate(["is_male", "is_service", "wfh_available"], start=2):
    test.insert(loc=loc, column=column, value=eval(column))

test.drop(columns=["Gender", "Company Type", "WFH Setup Available"], 
          axis=1, 
          inplace=True)

### 2.2.2 The Column: "Date of Joining"
All the employees in the dataset seem to join in 2008. Thus, omiting the year and converting the dates to integers (just to days) ***might*** help us a bit along the way. Here what we are going to do is:<br>
* **Remove the year**
* **Convert month + day to only days**
    * *Example-1:* join date of 01-20 (Jan 20th) will have $1*31 + 20 = 51$ days
    * *Example-2:* join date of 12-30 (Dec 30th) will have $2*31 + 30 = 402$ days
* **Inverse the sorting** because the smaller the integers are, the more experienced employee will be.
    * *Example:* So, the newest guy will have the maximum, the oldest will have the minimum days, right?
    * To inverse the situation, a simple calculation will be performed:
        * At the end, the oldest guy will have $|newGuysDays - oldGuysDays|$ in total w.r.t the new guy.
        * And the newest guy will have 0 days.
* Finally, **Normalize the numbers** to work with them better.<br>

**Note:** All these process would be in vain because it might not affect the model at all. We will see :)

In [ ]:
%%time
month_dict = {"01":31, "02":29, "03":31, 
              "04":30, "05":31, "06":30, 
              "07":31, "08":31, "09":30, 
              "10":31, "11":30, "12":31}

for dataset in (train, test):
    for i in range(len(dataset)):
        x = dataset["Date of Joining"][i].split("-")    # sample output: ['2008', '09', '30']
        x = eval(f"{int(x[1])} * {month_dict[x[1]]} + {int(x[2])}")    # sample output: 300
        dataset.loc[i, "Date of Joining"] = x

    dataset["Date of Joining"] = dataset["Date of Joining"].astype("float32")
    dataset["Date of Joining"] = abs(dataset["Date of Joining"] - dataset["Date of Joining"].max())
    dataset["Date of Joining"] /= dataset["Date of Joining"].max()

### 2.2.3 NaN Values

In [ ]:
# there are NaNs in the most critical variables
# let's dive into the details
resource_nan = train["Resource Allocation"].isna()
mental_nan = train["Mental Fatigue Score"].isna()
burnrate_nan = train["Burn Rate"].isna()

# No "Burn Rate" & "Mental Fatigue Score" values
train[(burnrate_nan) & (mental_nan)]

We can roughly predict the **Burn Rate** by **Mental Fatigue Score** (and vice-versa) but the absence of both of these values gives us a hard time and trying to predict the **Burn Rate** would be an example of overestimation. Thus we had better remove these double-NaN values to make things simpler.

In [ ]:
# Index values of NaN values of "Burn Rate" & "Mental Fatigue Score"
double_nan_indices = train[(burnrate_nan) & (mental_nan)].index

train.drop(index=double_nan_indices, axis=0, inplace=True)
train.reset_index(drop=True, inplace=True)
del double_nan_indices, is_male, is_service, wfh_available

In [ ]:
# okay, what else?
train.isna().sum()

Before we move on, it would be a wise practice to check the correlations of the variables to form a solid idea on what to do with the NaN values.

In [ ]:
train_corr = train.corr()

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(data=train_corr, 
            annot=True, 
            fmt=".2f", 
            vmin=-1, 
            vmax=1, 
            cmap="inferno")
plt.yticks(rotation=0)
plt.show()

In [ ]:
# We have very high correlation between some of the variables.
# Let's focus on "Mental Fatigue Score" and "Burn Rate"
plt.figure(figsize=(9, 5))
plt.scatter("Mental Fatigue Score", "Burn Rate", data=train, s=0.3, c="darkgreen")
plt.plot([0,10], [0, 1], linewidth=2, c="darkred")
plt.xlabel("Mental Fatigue Score", fontdict=axis_font)
plt.ylabel("Burn Rate", fontdict=axis_font)
plt.xticks(range(0,11))
plt.show()

The correlation is almost linear. How about we really *linearize* it and fit into our model because you know, models like to assume that the relationship between the features and target variables is linear. Then later, we may fill the gaps by using this high correlation by interpolating the points we have in our hand.

In [ ]:
LINEAR_CONST = 7 / 10

In [ ]:
fig, (ax1, ax2) = plt.subplots(1,2, sharex=True, sharey=True, figsize=(18 ,5))
fig.suptitle("Mental Fatigue Score vs. Burn Rate", color="black", size=22)

ax1.scatter("Mental Fatigue Score", 
            "Burn Rate", 
            data=train, 
            s=0.3, 
            c="darkgreen")
ax1.plot([0,10], [0, 1], linewidth=2, c="darkred")
ax1.set_title("Original Graph")
plt.xticks(range(0,11))

ax2.scatter(train["Mental Fatigue Score"], 
            train["Burn Rate"] ** LINEAR_CONST, 
            s=0.3, 
            c="green")
ax2.plot([0,10], [0, 1], linewidth=2, c="darkred")
ax2.set_title("Transformed Graph")

fig.text(x=0.09, y=0.5, 
         s="Burn Rate", 
         va="center", rotation="vertical", size=20, c="darkred")
fig.text(x=0.42, y=0.03, 
         s="Mental Fatigue Score", 
         va="center", rotation="horizontal", size=20, c="darkred")
plt.show()

After a few trial and error, we came up with a LINEAR_CONST of 0.7. Let's implement it to our datasets.

In [ ]:
# Note that we are not changing "Burn Out". It's our target variable. 
# We need to modify the fatigue score. Thus -1.
train["Mental Fatigue Score"] **= (LINEAR_CONST**-1)
test["Mental Fatigue Score"] **= (LINEAR_CONST**-1)

It's time to fill the holes.

In [ ]:
# to keep things safe, let's play on a copy of our train dataset
train_copy = pd.DataFrame()
for i in train.columns[-3:]:
    fill_with = train[i].interpolate(method="linear")
    train_copy[i] = train[i].fillna(fill_with, inplace=False)

plt.figure(figsize=(9, 5))
plt.scatter("Mental Fatigue Score", "Burn Rate", 
            data=train_copy, 
            s=0.3, 
            c="darkgreen")
plt.plot([0, train["Mental Fatigue Score"].max()], [0, 1], 
         linewidth=2, 
         c="darkred")
plt.xlabel("Mental Fatigue Score", fontdict=axis_font)
plt.ylabel("Burn Rate", fontdict=axis_font)
plt.show()
del train_copy

Random interpolation over the data made the dataset unnecessarily noisy. That why we should consider the correlations between the variables.<br>The things we are going to do are:<br>
* Create a copy of the dataset with *NaN-free* values on "Mental Fatigue Score" & "Burn Rate" variables
    * because the interpolation will calculate the relationship between them.
* Bake the interpolation function, i.e $y = f(x)$
* Predict the NaN values by the function and replace them with the new values

...but first, let's look at the original values for the last time:

In [ ]:
describe_original = train.describe()
describe_original

In [ ]:
from scipy import interpolate

In [ ]:
# Create the copy of our dataset and
# make sure that there are no NaN values in
# "Mental Fatigue Score" & "Burn Rate" variables

train_copy = train.copy(deep=True)

not_na1 = train_copy["Mental Fatigue Score"].notna()
not_na2 = train_copy["Burn Rate"].notna()

train_copy = train_copy[(not_na1) & (not_na2)]
del not_na2, not_na1

In [ ]:
# function to find the burn rate
fn_burn = interpolate.interp1d(x=train_copy["Mental Fatigue Score"], 
                               y=train_copy["Burn Rate"], 
                               kind="linear", 
                               fill_value=None)
# function to find the mental fatigue score
fn_mental = interpolate.interp1d(y=train_copy["Mental Fatigue Score"], 
                                 x=train_copy["Burn Rate"], 
                                 kind="linear", 
                                 fill_value=None)

In [ ]:
# get rid of these NaNs
for i in train[train["Burn Rate"].isna()].index:
    train.loc[i, "Burn Rate"] = fn_burn(train.loc[i, "Mental Fatigue Score"])
    
for i in train[train["Mental Fatigue Score"].isna()].index:
    train.loc[i, "Mental Fatigue Score"] = fn_mental(train.loc[i, "Burn Rate"])

In [ ]:
# well, what now?
train.isna().sum()

In [ ]:
# remember that "Resource Allocation" is also highly correlated with Designation.
plt.figure(figsize=(6, 5))
plt.scatter("Designation", "Resource Allocation", data=train, c="darkred")
plt.xlabel("Designation", fontdict=axis_font)
plt.ylabel("Resource Allocation", fontdict=axis_font)
plt.yticks(range(0,11))
plt.show()

In [ ]:
# We will get the mean of each "Designation" value and replace the NaN with them 
# Here, an example: 
temp_value = train["Resource Allocation"][train["Designation"]==1].mean()

print(f"The mean value of Resource Allocation where \
Designation == 1 is: {temp_value:.2f}")

In [ ]:
for i in range(6):    # 0 to 5, Designation values
    mean_value = train["Resource Allocation"][train["Designation"]==i].mean()
    
    # condition: "Designation" == i AND "Resource Allocation" is NaN
    condition = (train["Designation"]==i) & (train["Resource Allocation"].isna())
    
    # all NaN values are converted to mean values:
    train.loc[condition, "Resource Allocation"] = mean_value

del temp_value, train_copy, fn_burn, fn_mental

In [ ]:
# ...and it's gone!
train.isna().sum()

As can be seen below, standard deviations are almost the same. Yay!

In [ ]:
describe_botox = train.describe()

print("--------------- Original Data ---------------")
display(describe_original.iloc[:3, -3:])
print("\n--------------- Cleaned Data ---------------")
display(describe_botox.iloc[:3, -3:])

***

## 2.3 Bonus: A Fancy Way of EDA

This is a module that I've recently discovered so you may consider this section as a note to myself and skip it. As title stated, it's a "fancier" way of exploratory data analysis with an interactive GUI.

*Note-1: The module needs to be updated to be used on Colab.<br>Please use the command and restart the runtime:*

<code>!pip install -U pandas-profiling</code>

*Note-2: I commented out the codes because GitHub doesn't show the interactive stuff on its page even though it keeps the files on the cache. So please uncomment them and feel free to explore our cleaned data.*

Update & import the module:

In [ ]:
# !pip install -U pandas-profiling

In [ ]:
# from pandas_profiling import ProfileReport

**Profile Report**

In [ ]:
# profile = ProfileReport(train, title="Profiling Report", explorative=True)
# profile.to_notebook_iframe()

(Optional) Export the report as an html file:

In [ ]:
# profile.to_file("./burning_out_report.html")

***
# 3\. Training & Validation

## 3.1 Data Preparation

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
# 25% of the train set is split as a validation set
X_train, X_val, \
y_train, y_val = train_test_split(train.iloc[:, 1:-1],
                                         train.iloc[:, -1], 
                                         test_size=0.25, 
                                         shuffle=True, 
                                         random_state=19)
X_test = test.iloc[:, 1:]
y_test = None

print("Shape of the train set:\nX_train:", X_train.shape)
print("y_train:", y_train.shape)
print("\nShape of the validation set:\nX_val:", X_val.shape)
print("y_val:", y_val.shape)
print("\nShape of the test set:\nX_test:", X_test.shape)

In [ ]:
scale = StandardScaler()
normalize = MinMaxScaler((0, 1))

# scaled set: mean=0, standard deviation=1
X_train_std = scale.fit_transform(X_train)
X_val_std = scale.fit_transform(X_val)
X_test_std = scale.fit_transform(X_test)

# normalized set: values are between [0, 1]
X_train_norm = normalize.fit_transform(X_train)
X_val_norm = normalize.fit_transform(X_val)
X_test_norm = normalize.fit_transform(X_test)

## 3.2 Linear Regression Models

In [ ]:
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error as mae
from statsmodels.tools.eval_measures import mse, rmse

In [ ]:
# Let me create a function for error values
def show_errors(y_validations, y_predictions):
    """Function to show error statistics of the validation part.
    y_validations: Validation values
    y_predictions: Predicted values"""
    mae_fn = mae(y_validations, y_predictions)
    mse_fn = mse(y_validations, y_predictions)
    rmse_fn = rmse(y_validations, y_predictions)

    print("\n---------Error Statistics of Validation Part---------")
    print(f"Mean Absolute Error (MAE)             : {mae_fn:.4f}")
    print(f"Mean Square Error (MSE)               : {mse_fn:.4f}")
    print(f"Root Mean Square Error (RMSE)         : {rmse_fn:.4f}\n")

### 3.2.1 Ordinary Least Squares (As a baseline model)

In [ ]:
X_train_ols = sm.add_constant(X_train)
ols_results_model = sm.OLS(y_train, X_train_ols)
ols_results = ols_results_model.fit()

In [ ]:
ols_results.summary()

In [ ]:
# validation data
X_val_ols = sm.add_constant(X_val)
ols_results_val_model = sm.OLS(y_val, X_val_ols)
ols_results_val = ols_results_val_model.fit()
ols_results_val.summary()

In [ ]:
show_errors(y_val, 
            ols_results_val.predict(sm.add_constant(X_val_std))
           )

In [ ]:
def show_comparison(y_validations, y_predictions):
    """To see the comparison, please provide:
    y_validations and y_predictions"""
    plt.figure(figsize=(12,6))
    plt.scatter(y_validations, 
                y_predictions, 
                color="k", alpha=0.6, s=10, label="Predicted")
    plt.plot(y_validations, y_validations, "r--", label="True")
    plt.xlabel("True Values", fontdict=axis_font)
    plt.ylabel("Predicted Values", fontdict=axis_font)
    plt.title("True vs. Predicted Values", fontdict=title_font)
    plt.legend(fontsize="large", loc="best")
    plt.show()

In [ ]:
show_comparison(y_val, 
                ols_results_val.predict(sm.add_constant(X_val_std)))

We have an $adjusted \space R^2$ value of $0.919$, which seems nice but pretty bad error & prediction values as seen above. Also, as we predicted, **Date of Joining** and **Company Type** features are unfortunatelly not statically significant since **p-values** > 0.05 (including *gender* for the validation set). Before we move on to other models, let's check the linear regression assumptions for fun (or you can directly skip to the secion **3.2.2 Lasso Regression**). Some of these assumptions may be satisfied and some may not :( <br>Anyways, we will keep trying the other models and comment at the end. 

#### 3.2.1.1 Linear Regression Assumptions

In [ ]:
# Assumption: The relationship between the dependent (Y) and independent (X)
# variables should be linear.
# That's why we linearized the fatigue score to get a better result.

In [ ]:
# Assumption: Error term should be near zero in means
predict = ols_results.predict(X_train_ols)
errors = y_train - predict
print("The average error between the predicted & real value: \
{:.3g}".format(np.mean(errors)))

In [ ]:
# Assumption: Homoscedasticity 
# The error variance mustn't systematically change across the observations,
# i.e it should be the same for all the values of independent variables, X
plt.figure(figsize=(12,6))
plt.scatter(predict, errors, color="darkgreen", s=3, alpha=0.75)
plt.xlabel('Predicted Value', fontdict=axis_font)
plt.ylabel('Residuals', fontdict=axis_font)
plt.axhline(y=0, color="r")
plt.title('Residuals vs. Predict', fontdict=title_font)
plt.show()

Even though the variance seems almost constant, **Bartlett** and the **Levene** tests will confirm this assumption. The null hypothesis for both tests is that the errors are ***homoscedastic***. If the p-values < 0.05, the hypothesis will be rejected (i.e **heteroscedastic**).

In [ ]:
from scipy.stats import bartlett
from scipy.stats import levene

bart_stats = bartlett(predict, errors)
lev_stats = levene(predict, errors)

print("Bartlett test statistic value is {0:.2f} \
and p-value is {1:.3g}".format(bart_stats[0], bart_stats[1]))
print("Levene test statistic value is {0:.2f} \
and p-value is {1:.3g}".format(lev_stats[0], lev_stats[1]))

In [ ]:
# Assumption: Low multicollinearity
# We saw that there are high collinearities among the features :(

In [ ]:
# Assumption: Error terms should be uncorrelated with one another
plt.figure(figsize=(14,4))
plt.subplot(1,2,1)
plt.plot(errors, linewidth=0.1)

plt.subplot(1,2,2)
from statsmodels.tsa.stattools import acf
acf_data = acf(errors)
plt.plot(acf_data[1:], color="red")
plt.show()

From the first figure, we will see that the errors are uncorrelated where the second one implies very low autocorrelation among the errors (see the extremely low values). By the way, *autocorrelation function*, **acf()** computes the variables' correlation with each other.

In [ ]:
# Assumption: Features also shouldn't be correlated with the errors (exogeneity)
rand_nums = np.random.normal(np.mean(errors), 
                             np.std(errors), 
                             len(errors))
plt.figure(figsize=(14,6))
plt.subplot(1,2,1)
plt.scatter(rand_nums, errors, color="darkblue", s=3, alpha=0.75)
plt.xlabel("Normally Distributed Random Variables", fontdict=axis_font)
plt.ylabel("Errors of the Model", fontdict=axis_font)
plt.title("QQ plot", fontdict=title_font)

plt.subplot(1,2,2)
plt.hist(errors, bins=30, color="darkred")
plt.xlabel("Errors", fontdict=axis_font)
plt.title("Histogram of the Errors", fontdict=title_font)

plt.tight_layout()
plt.show()

If these assumptions don't hold, our predicted values would be biased. Thankfully, blue cloud shows us the independency between our variables and the errors. Also the histogram tells us whether the errors have normal distribution or not. It seems so but we need to check them statistically. **Jarque Bera** and **normal** tests tell us the truth. The null hypothesis of these tests are the errors being normally distributed.

In [ ]:
from scipy.stats import jarque_bera
from scipy.stats import normaltest
jb_stats = jarque_bera(errors)
norm_stats = normaltest(errors)

print("Jarque-Bera test statistics is {0:.2f} and \
p value is {1:.5g}".format(jb_stats[0], jb_stats[1]))
print("Normality test statistics is {0:.2f} and p \
value is {1:5g}".format(norm_stats[0], norm_stats[1]))

...and the null hypotheses are rejected. They do not have normal distributions. Shoot... ¯\\_(ツ)_/¯

Failure in these assumptions affect our estimations: OLS coefficients become biased, variance of predictions increases and less reliable model we face.

### 3.2.2 Lasso Regression

In [ ]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [ ]:
lasso_CV_model = LassoCV(alphas=np.logspace(-8, 6, num=30, base=10.0), 
                         cv=10, 
                         n_jobs=-1)
lasso_CV = lasso_CV_model.fit(X_train_std, y_train)

lasso_CV_score = lasso_CV.score(X_train_std, y_train)
print(f"Adjusted R-square value of train set: {lasso_CV_score:.3f}")
lasso_CV_score = lasso_CV.score(X_val_std, y_val)
print(f"Adjusted R-square value of validation set: {lasso_CV_score:.3f}")

show_errors(y_val, lasso_CV.predict(X_val_std))

### 3.2.3 Ridge Regression

In [ ]:
ridge_CV_model = RidgeCV(alphas=np.logspace(-8, 6, num=30, base=10.0), 
                         cv=10)
ridge_CV = ridge_CV_model.fit(X_train_std, y_train)

ridge_CV_score = ridge_CV.score(X_train_std, y_train)
print(f"Adjusted R-square value of train set: {ridge_CV_score:.3f}")
ridge_CV_score = ridge_CV.score(X_val_std, y_val)
print(f"Adjusted R-square value of validation set: {ridge_CV_score:.3f}")

show_errors(y_val, ridge_CV.predict(X_val_std))

### 3.2.4 ElasticNet Regression

In [ ]:
elasticNet_CV = ElasticNetCV(alphas=np.logspace(-6, 6, 30), 
                             l1_ratio=0.5, 
                             cv=10).fit(X_train_std, y_train)
elasticNet_score = elasticNet_CV.score(X_train_std, y_train)

elasticNet_CV_score = elasticNet_CV.score(X_train_std, y_train)
print(f"Adjusted R-square value of train set: {elasticNet_CV_score:.3f}")
elasticNet_CV_score = elasticNet_CV.score(X_val_std, y_val)
print(f"Adjusted R-square value of validation set: {elasticNet_CV_score:.3f}")

show_errors(y_val, elasticNet_CV.predict(X_val_std))

### 3.2.5 Lasso Regression with Polynomial Features

Since **Lasso** will do the feature selection for us *(remember the statistically insignificant features)*, it would be a wise choice to apply polynomial features over it.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
import warnings; warnings.filterwarnings("ignore")

poly = PolynomialFeatures(degree=4, 
                          interaction_only=False, 
                          order="F")
X_train_poly = poly.fit_transform(X_train_std)
X_val_poly = poly.fit_transform(X_val_std)

lasso_CV_poly = LassoCV(alphas=np.logspace(-8, 6, num=30, base=10.0), 
                        cv=10, 
                        n_jobs=-1).fit(X_train_poly, y_train)

lasso_score_poly = lasso_CV_poly.score(X_train_poly, y_train)
print(f"Adjusted R-square value of train set: {lasso_score_poly:.3f}")
lasso_score_poly = lasso_CV_poly.score(X_val_poly, y_val)
print(f"Adjusted R-square value of validation set: {lasso_score_poly:.3f}")

show_errors(y_val, lasso_CV_poly.predict(X_val_poly))

In [ ]:
show_comparison(y_val, lasso_CV_poly.predict(X_val_poly))

Let's keep the data with polynomial features. It seems quite okay. We submit the data and see what happens later.<br> ***(PS: We almost hit $92.7\% \space R^2$ value. I'm among the first 50 even at this point.)***

In [ ]:
import datetime
def export_output(algorithm, 
                  test_data, 
                  name=datetime.datetime.now().strftime("hour%H%M") + "_output"):
    """This function exports the predicted test data as a .csv file.
    Three input needs to be provided:
    1) The model you wish to use e.g "lasso_CV_poly",
    2) Predictions created by X_test, X_test_std or something similar,
    3) (optional) Name for the output e.g 'lasso_output'."""
    test_predict = algorithm.predict(test_data)
    predicted_output = pd.concat([pd.Series(test.iloc[:, 0]), 
                                            pd.Series(test_predict)],
                                 axis=1)
    wrong_column_name = predicted_output.columns[-1]
    predicted_output.rename(columns = {wrong_column_name:"Burn Rate"},
                            inplace=True)
    predicted_output.to_csv(path_or_buf=(name + ".csv"), 
                            index=False)
    predicted_output.head()

In [ ]:
export_output(lasso_CV_poly, poly.fit_transform(X_test_std), 
              name = "lassoPoly_output")

In [ ]:
# %whos    # to check who is getting in our way

In [ ]:
# Let's clean up the place before we continue.
del lasso_CV_score, ridge_CV_score, elasticNet_CV_score, poly, LassoCV, RidgeCV,\
    lasso_score_poly, warnings, jarque_bera, bartlett,\
    norm_stats, jb_stats, rand_nums, acf_data, X_val_ols, bart_stats,\
    lev_stats, predict, errors, X_train_ols, ols_results_model, ols_results,\
    ols_results_val_model, ols_results_val, lasso_CV_model, lasso_CV, ridge_CV,\
    ridge_CV_model, resource_nan, mental_nan, burnrate_nan, acf, ElasticNetCV,\
    month_dict, fill_with, elasticNet_CV, elasticNet_score, train_corr, levene,\

## 3.3 k-Nearest Neighbors

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
knn_model = KNeighborsRegressor(n_jobs=-1)
knn_params = {"n_neighbors":range(10,20), 
              "weights":("uniform", "distance"), 
              "metric":("euclidean", "minkowski", "manhattan")
              }

In [ ]:
def train_model(estimator, parameters, cv=10, verbose=1, X_data=X_train_std):
    """Function to train your model with a grid search.
    Just provide the function an estimator and its parameters.
    'estimator' example: knn_model
    'parameters' example: knn_params
    Optionally, you can change 'cv', 'verbose' or X_input as well."""
    gridCV = GridSearchCV(estimator=estimator, 
                          param_grid=parameters, 
                          n_jobs=-1, 
                          cv=cv, 
                          verbose=verbose)
    gridCV.fit(X_data, y_train)

    print("\nBest training parameters:", gridCV.best_params_)
    print("Best training score: {:.4f}".format(gridCV.best_score_))
    show_errors(y_val, gridCV.predict(X_val_std))
    return gridCV

In [ ]:
knn_gridCV = train_model(knn_model, knn_params)

In [ ]:
knn_results = pd.DataFrame(knn_gridCV.cv_results_)
knn_results = knn_results[["param_metric", "param_n_neighbors",
                           "param_weights", "mean_test_score"]]
knn_results = knn_results.sort_values(by="mean_test_score", ascending=False)
knn_results.head()

Close but not enough!

## 3.4 Decision Trees

In [ ]:
# import pydotplus, graphviz
# from IPython.display import Image
from sklearn.tree import DecisionTreeRegressor    #, export_graphviz

In [ ]:
tree_model = DecisionTreeRegressor(random_state=19)
tree_params = {"max_depth":range(8,10), 
               "min_samples_split":range(2,5), 
               "splitter":("best", "random"), 
               "min_samples_leaf":range(5,15)
              }

tree_gridCV = train_model(tree_model, tree_params)

In [ ]:
tree_results = pd.DataFrame(tree_gridCV.cv_results_)
tree_results = tree_results[["param_max_depth", 
                             "param_min_samples_split",
                             "param_splitter", 
                             "param_min_samples_leaf", 
                             "mean_test_score"]]
tree_results = tree_results.sort_values(by="mean_test_score", 
                                        ascending=False)
tree_results.head()

In [ ]:
export_output(algorithm = tree_gridCV, test_data = X_test_std,
              name = "decisionTree_output")

Decision tree results seem pretty good. Let's keep it as well.<br>
***PS: (After submission, we had a score slightly worse (~0.2%) than the Lasso model.)***

## 3.5 Random Forest

In [ ]:
import time
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, cross_val_predict

In [ ]:
# These values are from a series of everlasting grid searches.
forest_model = RandomForestRegressor(n_estimators=200, 
                                     max_depth=8, 
                                     criterion="mae", 
                                     min_samples_leaf=5, 
                                     min_samples_split=2, 
                                     random_state=19)

In [ ]:
forest_scoreCV = cross_val_score(estimator=forest_model, 
                                 X=X_train_std, 
                                 y=y_train, 
                                 cv=5, 
                                 verbose=2, 
                                 n_jobs=-1)

In [ ]:
print(f"Mean score of Random Forest model: {forest_scoreCV.mean():.3f}")

In [ ]:
forest_predictCV = cross_val_predict(estimator=forest_model, 
                                     X=X_val_std, 
                                     y=y_val, 
                                     cv=5, 
                                     verbose=2, 
                                     n_jobs=-1)

In [ ]:
show_errors(y_val, forest_predictCV)

We are still slightly behind **Lasso with polynomial features** model. The submitted score confirmed this :( Maybe a better hyperparameter tuning would succeed to the throne but it would take many hours to find out. Let's display the most significant variables to turn this all "random forest time waste" into something useful. So we can make sure which features have the most impact on our model.

In [ ]:
# Variable named 'sp' stands for 'significant pairs'
forest_model.fit(X_val_std, y_val)

sp = dict(zip(X_train.columns, forest_model.feature_importances_))
# sorting w.r.t their significance values:
sp = {k: v for k, v in sorted(sp.items(), key=lambda i: i[1], reverse=True)}

significant_names = list(sp.keys())
significant_values = list(sp.values())

significance = pd.Series(data=significant_values,
                         index=significant_names).sort_values()
plt.figure(dpi=75)
significance.plot(kind="barh", color="darkred")
plt.title("Significance of The Features")
plt.show()
del sp, significant_names, significant_values, significance

## 3.6 Support Vector Machines

In [ ]:
from sklearn.svm import SVR

In [ ]:
time_start = time.time()
svm_model = SVR(C=0.01, kernel="linear")    # grid searched prior to this
svm_params = {"epsilon":np.arange(0.01, 0.1, 0.02)}

svm_gridCV = train_model(svm_model, svm_params, cv=5)
print("\nTraining is completed in {:.2f} seconds".format(time.time() - time_start))

In [ ]:
svm_results = pd.DataFrame(svm_gridCV.cv_results_)
svm_results = svm_results[["param_epsilon", "mean_test_score"]]
svm_results = svm_results.sort_values(by="mean_test_score", 
                                            ascending=False)
svm_results.head()

I have high hopes for XGBoost.

## 3.7 XGBoost

In [ ]:
import xgboost as xgb

In [ ]:
xgb_model = xgb.XGBRegressor(booster="gbtree", 
                             gamma=0, 
                             max_depth=6, 
                             alpha=0, 
                             eta=0.1)
# again, after a series of grid search, this one is the last one:
xgb_params = {"objective":["reg:squarederror"],
              "n_estimators":[47], 
              "min_child_weight":[7]}

xgb_gridCV = train_model(xgb_model, xgb_params, cv=20, verbose=1)

In [ ]:
xgb_results = pd.DataFrame(xgb_gridCV.cv_results_)
xgb_results = xgb_results[["param_objective",  
                           "param_n_estimators", 
                           "param_min_child_weight", 
                           "mean_test_score"]]
xgb_results = xgb_results.sort_values(by="mean_test_score", 
                                            ascending=False)
xgb_results.head()

In [ ]:
# let's save this model too
export_output(xgb_gridCV, X_test_std, name="xgb_output")

XGBoost is the winner at this point. We improved our score and hit almost $92.9\% \space R^2$ value on the submission.

In [ ]:
# feature importance
xgb_model2 = xgb.train(params=xgb_gridCV.best_params_, 
                      dtrain=xgb.DMatrix(data=X_train_std, label=y_train))
ax = xgb.plot_importance(xgb_model2)
ax.figure.set_size_inches(10, 8)
plt.show()

XGBoost seems to use our features more efficiently. We had a similar significance graph as random forest.<br>Now comes my favorite part: *Deep Learning* algorithm!<br>I'm not sure if we can beat XGBoost or not but we surely will try hard!

***
## 3.8 Artificial Neural Networks

In [ ]:
from keras import models, layers
from keras.optimizers import Adam, schedules

In [ ]:
# to avoid repeating the layers
def dense_layer(unit):
    model.add(layers.Dense(units=unit))
    model.add(layers.BatchNormalization())
    model.add(layers.Activation("relu"))

Just one hidden layer with 16 neurons did the job without an overfit. (Note that we put the BatchNormalization in between to normalize the transition from linearity (dense layer) to non-linearity (activation function).

In [ ]:
model = models.Sequential()
model.add(layers.Dense(units=16,
                       input_shape=(X_train_norm.shape[1], ), 
                       kernel_initializer="GlorotUniform", 
                       name="Hidden_Layer1"))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))
#dense_layer(64)
model.add(layers.Dropout(0.05))
#dense_layer(128)
model.add(layers.Dense(1, name="Output_Layer"))
model.summary()

In [ ]:
lr_schedule = schedules.ExponentialDecay(initial_learning_rate=1e-3,
                                         decay_steps=10000,
                                         decay_rate=0.9)

opt = Adam(learning_rate=lr_schedule)
model.compile(optimizer=opt, loss="mse", metrics=["mae"])
model_history = model.fit(X_train_norm, y_train, 
                          epochs=30, 
                          batch_size=32, 
                          validation_data=(X_val_norm, y_val), 
                          shuffle=False, 
                          verbose=0)

In [ ]:
show_errors(y_val, model.predict(X_val_norm).reshape(-1))

We need no more than 30 epochs. It's a waste of time. Trust me, I tried.

In [ ]:
history_dict = model_history.history

train_loss = history_dict["loss"]
val_loss = history_dict["val_loss"]
train_mae = history_dict["mae"]
val_mae = history_dict["val_mae"]

In [ ]:
plt.figure(figsize=(20,8))

plt.subplot(1,2,1)
plt.plot(range(1, len(train_mae)+1), train_mae, 
         color="darkgreen", linewidth=2, label="Train Set")

plt.plot(range(1, len(val_mae)+1), val_mae, 
        "r--", linewidth=2, label="Valdation Set")

plt.xlabel("Epochs", fontdict=axis_font)
plt.ylabel("Mean Absolute Error", fontdict=axis_font)
plt.legend(fontsize="medium", loc=(0.7,0.5))

plt.subplot(1,2,2)
plt.plot(range(1, len(train_loss)+1), train_loss, 
         color="darkgreen", linewidth=2, label="Train Set")

plt.plot(range(1, len(val_loss)+1), val_loss, 
        "r--", linewidth=2, label="Valdation Set")

plt.xlabel("Epochs", fontdict=axis_font)
plt.ylabel("Loss Value", fontdict=axis_font)
plt.legend(fontsize="medium", loc=(0.7,0.5))

plt.show()

***

# 4\. Final Remarks

1) I have a confession to make: If we skipped the whole NaN value modification and just killed them all with <code>pandas.DataFrame.dropna()</code> function, we would have had better results in all of the models including our *champ* XGBoost. Removing all the NaNs and running the dataset on XGBoost gave me 92.98% submission score, slightly higher than the former but unfortunatelly it's not the one I submitted before the deadline. These are the best XGBoost scores from this notebook (You saw the 1st one. 2nd is the result of <code>.dropna()</code> trick):


![The Winner XGBOOST](https://i.pinimg.com/originals/4e/e6/ff/4ee6ff59edbbcab85a462002435f1ef6.png)

2) Yes, **XGBoost** won the game. I couldn't manage to go beyond 93% **(to be exact: 92.89% with NaN transformation, 92.98% with NaN slaughter)** while the first place of the leaderboard is occupied by a score of 93.12%. Yeah, sometimes you all need a 0.14% bump and you just can't make it. The hackathon has come to an end as I'm writing this and got the 58th place -*48th with that* <code>.dropna()</code> thing- *(out of 500+ attendees, which doesn't seem bad)*. That little difference got me fall behind 50+ people. I also tried some feature engineering (as *Polynomial Feature* function did by itself up to 4th degree) by hand but no avail.

3) Remember the insignificant features? The score also changed a little when they were out. For this dataset, the roadmap should have been this way:<br>
* Remove all the NaN values.
* Perform one-hot encoding
* Try some feature engineering
* Get rid of insignificant features (because they unnecessarily occupy weights as well)
* Determine the best models
* Hyperparameter tuning of best models

$92.98\% $ is the score that I got after these steps (after the deadline).

4) Obviously, there is still some room to make improvements over those bullet points but for me, that's all folks! This was an easy dataset since "Mental Fatigue Score" dominated our prediction process and left little room for other features but I hope this notebook shines a little bit of light on the things you didn't know before. Wish you a better luck than I had! :) If you think there seems a better way (another model, a different hyperparameter or an interesting way of feature engineering), let me know. We are all here in this world to learn!